In [0]:
%pip install lyricsgenius

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, FloatType, BooleanType
from pyspark import SparkFiles

In [0]:
# Access token to access the Genius API
genius_access_token = "_uAHk5SWEsiQ-m07ZfEkAINUaD8Qlu497ipRp3MUD-fDVw1MpfEPeY5nx3kg08pB"

from lyricsgenius import Genius

# Re-usable object to search and save lyrics.
genius = Genius(genius_access_token)


In [0]:
# Let's try the Genius API to see if it works as expected:
song = genius.search_song('Hey, Ma', 'Bon Iver')
print(song.lyrics)

Searching for "Hey, Ma" by Bon Iver...
Done.
57 ContributorsHey, Ma Lyrics[Verse 1]
I waited outside
I took it remote
I wanted a bath
"Tell the story or he goes"
"Tell the story or he goes"

[Chorus]
Full time, you talk your money up
While it's living in a coal mine
Tall time to call your Ma
Hey Ma, hey Ma
Tall vote, you know you mope it up
Well, you wanted it your whole life
You're back and forth with light

[Verse 2]
I waited outside
I was tokin' on dope
I hoped it all wunt go in a minute
With the past that you know
I wanted all that mind sugar
I want it all mine
I had a heavy mind, sugar
I took it, it was right
You might also like[Chorus]
Full time, you talk your money up
While it's living in a coal mine
Tall time to call your Ma
Hey Ma, hey Ma
Tall vote, you know you mope it up
Well, you wanted it your whole life
You're back and forth with light

[Interlude]
You're back and forth
You're back and forth
You go back and forth
And then you're back and forth
And back and forth

[Verse 3

In [0]:
# Let's store the file in dbfs, if it's not there, fetch it and save it to dbfs
def getGrammyData():
    file_location = "/FileStore/tables/the_grammy_awards.csv"

    try:
        f = dbutils.fs.ls(file_location)
    except:
        url = "https://raw.githubusercontent.com/mackrauss/3252-BigData-Group-F/main/the_grammy_awards.csv"
        spark.sparkContext.addFile(url)
        dbutils.fs.cp('file://'+SparkFiles.get('the_grammy_awards.csv'), file_location)
        print('Moved file to tables')
    
    return file_location

In [0]:
file_location = getGrammyData()

file_type = "csv"

# CSV options
infer_schema = "false"
first_row_is_header = "true"
delimiter = ","

# Adding 'escape' option to change escape character from \ to " - not we need to escape the " inside the
# see https://stackoverflow.com/questions/40413526/reading-csv-files-with-quoted-fields-containing-embedded-commas
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .option("escape", "\"") \
  .load(file_location)

In [0]:
# pull file via HTTP/S using Spark tools
# from pyspark import SparkFiles

# url = "https://raw.githubusercontent.com/mackrauss/3252-BigData-Group-F/main/the_grammy_awards.csv"
# spark.sparkContext.addFile(url)

# df = spark.read.csv("file://"+SparkFiles.get("the_grammy_awards.csv"), header=True, inferSchema= True)


In [0]:
display(df.head(5))

year,title,published_at,updated_at,category,nominee,artist,workers,img,winner
2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,Record Of The Year,Bad Guy,Billie Eilish,"Finneas O'Connell, producer; Rob Kinelski & Finneas O'Connell, engineers/mixers; John Greenham, mastering engineer",https://www.grammy.com/sites/com/files/styles/artist_circle/public/muzooka/Billie%2BEilish/Billie%2520Eilish_1_1_1594138954.jpg?itok=3-71Dfxh,True
2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,Record Of The Year,"Hey, Ma",Bon Iver,"BJ Burton, Brad Cook, Chris Messina & Justin Vernon, producers; BJ Burton, Zach Hanson & Chris Messina, engineers/mixers; Greg Calbi, mastering engineer",https://www.grammy.com/sites/com/files/styles/artist_circle/public/muzooka/Bon%2BIver/Bon%2520Iver_1_1_1578385181.jpg?itok=_M1hc5Ux,True
2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,Record Of The Year,7 rings,Ariana Grande,"Charles Anderson, Tommy Brown, Michael Foster & Victoria Monet, producers; Serban Ghenea, John Hanes, Billy Hickey & Brendan Morawski, engineers/mixers; Randy Merrill, mastering engineer",https://www.grammy.com/sites/com/files/styles/artist_circle/public/muzooka/Ariana%2BGrande/Ariana%2520Grande_1_1_1578384678.jpg?itok=Amj90j13,True
2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,Record Of The Year,Hard Place,H.E.R.,"Rodney “Darkchild” Jerkins, producer; Joseph Hurtado, Jaycen Joshua, Derek Keota & Miki Tsutsumi, engineers/mixers; Colin Leonard, mastering engineer",https://www.grammy.com/sites/com/files/styles/artist_circle/public/muzooka/H.E.R./H.E.R._1_1_1594631035.jpg?itok=ClJe-2MN,True
2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,Record Of The Year,Talk,Khalid,"Disclosure & Denis Kosiak, producers; Ingmar Carlson, Jon Castelli, Josh Deguzman, John Kercy, Denis Kosiak, Guy Lawrence & Michael Romero, engineers/mixers; Dale Becker, mastering engineer",https://www.grammy.com/sites/com/files/styles/artist_circle/public/muzooka/Khalid/Khalid_1_1_1594578772.jpg?itok=2HxjAT-X,True


In [0]:
# Create a view or table to allow SQL usage

temp_table_name = "grammy_awards"

df.createOrReplaceTempView(temp_table_name)

In [0]:
# Show all years in the record
df.select("year").distinct().orderBy(df.year.desc()).show() 

+----+
|year|
+----+
|2019|
|2018|
|2017|
|2016|
|2015|
|2014|
|2013|
|2012|
|2011|
|2010|
|2009|
|2008|
|2007|
|2006|
|2005|
|2004|
|2003|
|2002|
|2001|
|2000|
+----+
only showing top 20 rows



In [0]:
# Count number of years recorded
df.select("year").distinct().count()

Out[10]: 62

In [0]:
%sql
-- count number of years
select count(distinct year) AS `# years recorded`
from `grammy_awards`

# years recorded
62


In [0]:
%sql
-- count number of categories
select count(distinct category) `# of categories`
from `grammy_awards`

# of categories
638


In [0]:
%sql
-- ensure we have 'Record Of The Year' data for each year
select count(distinct year) as `Years with Record of the Year recorded`
from `grammy_awards`
where category like 'Record Of The Year'


Years with Record of the Year recorded
62


In [0]:
# Create a new DataFrame that only contains the 'Record Of The Year' category data
df_record = df.select('year','title','category','nominee','artist','workers','img','winner').where(df.category == "Record Of The Year")
display(df_record.head(5))

year,title,category,nominee,artist,workers,img,winner
2019,62nd Annual GRAMMY Awards (2019),Record Of The Year,Bad Guy,Billie Eilish,"Finneas O'Connell, producer; Rob Kinelski & Finneas O'Connell, engineers/mixers; John Greenham, mastering engineer",https://www.grammy.com/sites/com/files/styles/artist_circle/public/muzooka/Billie%2BEilish/Billie%2520Eilish_1_1_1594138954.jpg?itok=3-71Dfxh,True
2019,62nd Annual GRAMMY Awards (2019),Record Of The Year,"Hey, Ma",Bon Iver,"BJ Burton, Brad Cook, Chris Messina & Justin Vernon, producers; BJ Burton, Zach Hanson & Chris Messina, engineers/mixers; Greg Calbi, mastering engineer",https://www.grammy.com/sites/com/files/styles/artist_circle/public/muzooka/Bon%2BIver/Bon%2520Iver_1_1_1578385181.jpg?itok=_M1hc5Ux,True
2019,62nd Annual GRAMMY Awards (2019),Record Of The Year,7 rings,Ariana Grande,"Charles Anderson, Tommy Brown, Michael Foster & Victoria Monet, producers; Serban Ghenea, John Hanes, Billy Hickey & Brendan Morawski, engineers/mixers; Randy Merrill, mastering engineer",https://www.grammy.com/sites/com/files/styles/artist_circle/public/muzooka/Ariana%2BGrande/Ariana%2520Grande_1_1_1578384678.jpg?itok=Amj90j13,True
2019,62nd Annual GRAMMY Awards (2019),Record Of The Year,Hard Place,H.E.R.,"Rodney “Darkchild” Jerkins, producer; Joseph Hurtado, Jaycen Joshua, Derek Keota & Miki Tsutsumi, engineers/mixers; Colin Leonard, mastering engineer",https://www.grammy.com/sites/com/files/styles/artist_circle/public/muzooka/H.E.R./H.E.R._1_1_1594631035.jpg?itok=ClJe-2MN,True
2019,62nd Annual GRAMMY Awards (2019),Record Of The Year,Talk,Khalid,"Disclosure & Denis Kosiak, producers; Ingmar Carlson, Jon Castelli, Josh Deguzman, John Kercy, Denis Kosiak, Guy Lawrence & Michael Romero, engineers/mixers; Dale Becker, mastering engineer",https://www.grammy.com/sites/com/files/styles/artist_circle/public/muzooka/Khalid/Khalid_1_1_1594578772.jpg?itok=2HxjAT-X,True


In [0]:
# Create a view or table for Record Of the Year data

temp_table_name = "record_of_the_year"

df_record.createOrReplaceTempView(temp_table_name)

In [0]:
# The next code block shows a data import error for 'Old Town Road' where escape character lead to more columns and thus no URL in the img filed
# Use this to export a CSV with all "Record of the Year" rows and do a quick check to ensure the correct data ends up in each column
# Armin 17.09.2023 Adding .option("escape", "\"") to the file import seems to have fixed the issue around img
# https://stackoverflow.com/questions/40413526/reading-csv-files-with-quoted-fields-containing-embedded-commas
display(df_record)

year,title,category,nominee,artist,workers,img,winner
2019,62nd Annual GRAMMY Awards (2019),Record Of The Year,Bad Guy,Billie Eilish,"Finneas O'Connell, producer; Rob Kinelski & Finneas O'Connell, engineers/mixers; John Greenham, mastering engineer",https://www.grammy.com/sites/com/files/styles/artist_circle/public/muzooka/Billie%2BEilish/Billie%2520Eilish_1_1_1594138954.jpg?itok=3-71Dfxh,True
2019,62nd Annual GRAMMY Awards (2019),Record Of The Year,"Hey, Ma",Bon Iver,"BJ Burton, Brad Cook, Chris Messina & Justin Vernon, producers; BJ Burton, Zach Hanson & Chris Messina, engineers/mixers; Greg Calbi, mastering engineer",https://www.grammy.com/sites/com/files/styles/artist_circle/public/muzooka/Bon%2BIver/Bon%2520Iver_1_1_1578385181.jpg?itok=_M1hc5Ux,True
2019,62nd Annual GRAMMY Awards (2019),Record Of The Year,7 rings,Ariana Grande,"Charles Anderson, Tommy Brown, Michael Foster & Victoria Monet, producers; Serban Ghenea, John Hanes, Billy Hickey & Brendan Morawski, engineers/mixers; Randy Merrill, mastering engineer",https://www.grammy.com/sites/com/files/styles/artist_circle/public/muzooka/Ariana%2BGrande/Ariana%2520Grande_1_1_1578384678.jpg?itok=Amj90j13,True
2019,62nd Annual GRAMMY Awards (2019),Record Of The Year,Hard Place,H.E.R.,"Rodney “Darkchild” Jerkins, producer; Joseph Hurtado, Jaycen Joshua, Derek Keota & Miki Tsutsumi, engineers/mixers; Colin Leonard, mastering engineer",https://www.grammy.com/sites/com/files/styles/artist_circle/public/muzooka/H.E.R./H.E.R._1_1_1594631035.jpg?itok=ClJe-2MN,True
2019,62nd Annual GRAMMY Awards (2019),Record Of The Year,Talk,Khalid,"Disclosure & Denis Kosiak, producers; Ingmar Carlson, Jon Castelli, Josh Deguzman, John Kercy, Denis Kosiak, Guy Lawrence & Michael Romero, engineers/mixers; Dale Becker, mastering engineer",https://www.grammy.com/sites/com/files/styles/artist_circle/public/muzooka/Khalid/Khalid_1_1_1594578772.jpg?itok=2HxjAT-X,True
2019,62nd Annual GRAMMY Awards (2019),Record Of The Year,Old Town Road,Lil Nas X Featuring Billy Ray Cyrus,"Andrew ""VoxGod"" Bolooki, Jocelyn “Jozzy” Donald & YoungKio, producers; Andrew ""VoxGod"" Bolooki, Cinco & Joe Grasso, engineers/mixers; Eric Lagg, mastering engineer",https://www.grammy.com/sites/com/files/styles/artist_circle/public/muzooka/Montero%2BLamar%2BHill/Lil%2520Nas%2520X_1_1_1581656760.jpg?itok=_YR6dOjI,True
2019,62nd Annual GRAMMY Awards (2019),Record Of The Year,Truth Hurts,Lizzo,"Ricky Reed & Tele, producers; Chris Galland, Manny Marroquin & Ethan Shumaker, engineers/mixers; Chris Gehringer, mastering engineer",https://www.grammy.com/sites/com/files/styles/artist_circle/public/muzooka/Lizzo/Lizzo_1_1_1583931631.jpg?itok=Gm2FQx4s,True
2019,62nd Annual GRAMMY Awards (2019),Record Of The Year,Sunflower,Post Malone & Swae Lee,"Louis Bell & Carter Lang, producers; Louis Bell & Manny Marroquin, engineers/mixers; Mike Bozzi, mastering engineer",https://www.grammy.com/sites/com/files/styles/artist_circle/public/muzooka/Post%2BMalone/Post%2520Malone_1_1_1594910341.jpg?itok=HCyRm13g,True
2018,61st Annual GRAMMY Awards (2018),Record Of The Year,This Is America,Childish Gambino,"Donald Glover & Ludwig Göransson, producers; Derek ""MixedByAli"" Ali, Kesha Lee, Riley Mackin, Shaan Singh & Alex Tumay, engineers/mixers; Mike Bozzi, mastering engineer",https://www.grammy.com/sites/com/files/styles/artist_circle/public/muzooka/Childish%2BGambino/Childish%2520Gambino_1_1_1591112936.jpg?itok=AU_J_f8x,True
2017,60th Annual GRAMMY Awards (2017),Record Of The Year,24K Magic,Bruno Mars,"Shampoo Press & Curl (Christopher Brody Brown, Philip Lawrence & Bruno Mars), producers; Serban Ghenea, John Hanes & Charles Moniz, engineers/mixers; Tom Coyne, mastering engineer",https://www.grammy.com/sites/com/files/styles/artist_circle/public/muzooka/Bruno%2BMars/Bruno%2520Mars_1_1_1578385368.jpg?itok=KhxnacRR,True


In [0]:
%sql
-- Show data import error
-- if the field img shows an URL the import worked as expected
select nominee, artist, workers, img, winner
from `record_of_the_year`
where nominee = "Old Town Road"

nominee,artist,workers,img,winner
Old Town Road,Lil Nas X Featuring Billy Ray Cyrus,"Andrew ""VoxGod"" Bolooki, Jocelyn “Jozzy” Donald & YoungKio, producers; Andrew ""VoxGod"" Bolooki, Cinco & Joe Grasso, engineers/mixers; Eric Lagg, mastering engineer",https://www.grammy.com/sites/com/files/styles/artist_circle/public/muzooka/Montero%2BLamar%2BHill/Lil%2520Nas%2520X_1_1_1581656760.jpg?itok=_YR6dOjI,True


In [0]:
%sql
select *
from `record_of_the_year`

-- order by year

year,title,category,nominee,artist,workers,img,winner
2019,62nd Annual GRAMMY Awards (2019),Record Of The Year,Bad Guy,Billie Eilish,"Finneas O'Connell, producer; Rob Kinelski & Finneas O'Connell, engineers/mixers; John Greenham, mastering engineer",https://www.grammy.com/sites/com/files/styles/artist_circle/public/muzooka/Billie%2BEilish/Billie%2520Eilish_1_1_1594138954.jpg?itok=3-71Dfxh,True
2019,62nd Annual GRAMMY Awards (2019),Record Of The Year,"Hey, Ma",Bon Iver,"BJ Burton, Brad Cook, Chris Messina & Justin Vernon, producers; BJ Burton, Zach Hanson & Chris Messina, engineers/mixers; Greg Calbi, mastering engineer",https://www.grammy.com/sites/com/files/styles/artist_circle/public/muzooka/Bon%2BIver/Bon%2520Iver_1_1_1578385181.jpg?itok=_M1hc5Ux,True
2019,62nd Annual GRAMMY Awards (2019),Record Of The Year,7 rings,Ariana Grande,"Charles Anderson, Tommy Brown, Michael Foster & Victoria Monet, producers; Serban Ghenea, John Hanes, Billy Hickey & Brendan Morawski, engineers/mixers; Randy Merrill, mastering engineer",https://www.grammy.com/sites/com/files/styles/artist_circle/public/muzooka/Ariana%2BGrande/Ariana%2520Grande_1_1_1578384678.jpg?itok=Amj90j13,True
2019,62nd Annual GRAMMY Awards (2019),Record Of The Year,Hard Place,H.E.R.,"Rodney “Darkchild” Jerkins, producer; Joseph Hurtado, Jaycen Joshua, Derek Keota & Miki Tsutsumi, engineers/mixers; Colin Leonard, mastering engineer",https://www.grammy.com/sites/com/files/styles/artist_circle/public/muzooka/H.E.R./H.E.R._1_1_1594631035.jpg?itok=ClJe-2MN,True
2019,62nd Annual GRAMMY Awards (2019),Record Of The Year,Talk,Khalid,"Disclosure & Denis Kosiak, producers; Ingmar Carlson, Jon Castelli, Josh Deguzman, John Kercy, Denis Kosiak, Guy Lawrence & Michael Romero, engineers/mixers; Dale Becker, mastering engineer",https://www.grammy.com/sites/com/files/styles/artist_circle/public/muzooka/Khalid/Khalid_1_1_1594578772.jpg?itok=2HxjAT-X,True
2019,62nd Annual GRAMMY Awards (2019),Record Of The Year,Old Town Road,Lil Nas X Featuring Billy Ray Cyrus,"Andrew ""VoxGod"" Bolooki, Jocelyn “Jozzy” Donald & YoungKio, producers; Andrew ""VoxGod"" Bolooki, Cinco & Joe Grasso, engineers/mixers; Eric Lagg, mastering engineer",https://www.grammy.com/sites/com/files/styles/artist_circle/public/muzooka/Montero%2BLamar%2BHill/Lil%2520Nas%2520X_1_1_1581656760.jpg?itok=_YR6dOjI,True
2019,62nd Annual GRAMMY Awards (2019),Record Of The Year,Truth Hurts,Lizzo,"Ricky Reed & Tele, producers; Chris Galland, Manny Marroquin & Ethan Shumaker, engineers/mixers; Chris Gehringer, mastering engineer",https://www.grammy.com/sites/com/files/styles/artist_circle/public/muzooka/Lizzo/Lizzo_1_1_1583931631.jpg?itok=Gm2FQx4s,True
2019,62nd Annual GRAMMY Awards (2019),Record Of The Year,Sunflower,Post Malone & Swae Lee,"Louis Bell & Carter Lang, producers; Louis Bell & Manny Marroquin, engineers/mixers; Mike Bozzi, mastering engineer",https://www.grammy.com/sites/com/files/styles/artist_circle/public/muzooka/Post%2BMalone/Post%2520Malone_1_1_1594910341.jpg?itok=HCyRm13g,True
2018,61st Annual GRAMMY Awards (2018),Record Of The Year,This Is America,Childish Gambino,"Donald Glover & Ludwig Göransson, producers; Derek ""MixedByAli"" Ali, Kesha Lee, Riley Mackin, Shaan Singh & Alex Tumay, engineers/mixers; Mike Bozzi, mastering engineer",https://www.grammy.com/sites/com/files/styles/artist_circle/public/muzooka/Childish%2BGambino/Childish%2520Gambino_1_1_1591112936.jpg?itok=AU_J_f8x,True
2017,60th Annual GRAMMY Awards (2017),Record Of The Year,24K Magic,Bruno Mars,"Shampoo Press & Curl (Christopher Brody Brown, Philip Lawrence & Bruno Mars), producers; Serban Ghenea, John Hanes & Charles Moniz, engineers/mixers; Tom Coyne, mastering engineer",https://www.grammy.com/sites/com/files/styles/artist_circle/public/muzooka/Bruno%2BMars/Bruno%2520Mars_1_1_1578385368.jpg?itok=KhxnacRR,True


In [0]:
df_record = df.select('year','title','category','nominee','artist','winner').where(df.category == "Record Of The Year")
display(df_record.head(5))

## Why are all of the records winners?

year,title,category,nominee,artist,winner
2019,62nd Annual GRAMMY Awards (2019),Record Of The Year,Bad Guy,Billie Eilish,True
2019,62nd Annual GRAMMY Awards (2019),Record Of The Year,"Hey, Ma",Bon Iver,True
2019,62nd Annual GRAMMY Awards (2019),Record Of The Year,7 rings,Ariana Grande,True
2019,62nd Annual GRAMMY Awards (2019),Record Of The Year,Hard Place,H.E.R.,True
2019,62nd Annual GRAMMY Awards (2019),Record Of The Year,Talk,Khalid,True


In [0]:
# Size of the data set?
display(df.count())

4810

In [0]:
import pandas as pd
# Reduce data set for some quick testing
record_list = df_record.take(8)
df_record_new = pd.DataFrame(record_list, columns=['year','title','category','nominee','artist','winner'])
display(df_record_new)

year,title,category,nominee,artist,winner
2019,62nd Annual GRAMMY Awards (2019),Record Of The Year,Bad Guy,Billie Eilish,True
2019,62nd Annual GRAMMY Awards (2019),Record Of The Year,"Hey, Ma",Bon Iver,True
2019,62nd Annual GRAMMY Awards (2019),Record Of The Year,7 rings,Ariana Grande,True
2019,62nd Annual GRAMMY Awards (2019),Record Of The Year,Hard Place,H.E.R.,True
2019,62nd Annual GRAMMY Awards (2019),Record Of The Year,Talk,Khalid,True
2019,62nd Annual GRAMMY Awards (2019),Record Of The Year,Old Town Road,Lil Nas X Featuring Billy Ray Cyrus,True
2019,62nd Annual GRAMMY Awards (2019),Record Of The Year,Truth Hurts,Lizzo,True
2019,62nd Annual GRAMMY Awards (2019),Record Of The Year,Sunflower,Post Malone & Swae Lee,True


In [0]:
import time
# Let's try to get one of the lyrics to see if the Genius API works
def get_lyrics (artist, song_name):
    song = genius.search_song(artist, song_name)
    # time.sleep(1)
    return song.lyrics
    # print(song.lyrics)

In [0]:
lyrics = get_lyrics('Hey, Ma', 'Bon Iver')
print (lyrics)

Searching for "Hey, Ma" by Bon Iver...
Done.
57 ContributorsHey, Ma Lyrics[Verse 1]
I waited outside
I took it remote
I wanted a bath
"Tell the story or he goes"
"Tell the story or he goes"

[Chorus]
Full time, you talk your money up
While it's living in a coal mine
Tall time to call your Ma
Hey Ma, hey Ma
Tall vote, you know you mope it up
Well, you wanted it your whole life
You're back and forth with light

[Verse 2]
I waited outside
I was tokin' on dope
I hoped it all wunt go in a minute
With the past that you know
I wanted all that mind sugar
I want it all mine
I had a heavy mind, sugar
I took it, it was right
You might also like[Chorus]
Full time, you talk your money up
While it's living in a coal mine
Tall time to call your Ma
Hey Ma, hey Ma
Tall vote, you know you mope it up
Well, you wanted it your whole life
You're back and forth with light

[Interlude]
You're back and forth
You're back and forth
You go back and forth
And then you're back and forth
And back and forth

[Verse 3

In [0]:
def test_func (artist, song_name):
    print(artist + ' ' + song_name)

In [0]:
# data['BMI'] = data.apply(lambda x: calc_bmi(x['Weight'], x['Height']), axis=1)
df_record_new['lyrics'] = df_record_new.apply(lambda x: get_lyrics(x['artist'], x['nominee']), axis=1)

Searching for "Billie Eilish" by Bad Guy...
Done.
Searching for "Bon Iver" by Hey, Ma...
Done.
Searching for "Ariana Grande" by 7 rings...
Done.
Searching for "H.E.R." by Hard Place...
Done.
Searching for "Khalid" by Talk...
Done.
Searching for "Lil Nas X Featuring Billy Ray Cyrus" by Old Town Road...
Done.
Searching for "Lizzo" by Truth Hurts...
Done.
Searching for "Post Malone & Swae Lee" by Sunflower...
Done.


In [0]:
display(df_record_new)

year,title,category,nominee,artist,winner,lyrics
2019,62nd Annual GRAMMY Awards (2019),Record Of The Year,Bad Guy,Billie Eilish,True,"344 ContributorsTranslationsHrvatskiTürkçeSvenskaEspañolРусскийRomânăPortuguêsPolskiItalianoHebrewDeutschFrançaisفارسیNederlandsDanskChinese TraditionalΕλληνικάالعربيةไทย / Phasa Thai​bad guy Lyrics[Verse 1] White shirt now red, my bloody nose Sleepin', you're on your tippy toes Creepin' around like no one knows Think you're so criminal Bruises on both my knees for you Don't say thank you or please I do what I want when I'm wanting to My soul? So cynical [Chorus] So you're a tough guy Like it really rough guy Just can't get enough guy Chest always so puffed guy I'm that bad type Make your mama sad type Make your girlfriend mad tight Might seduce your dad type I'm the bad guy Duh [Post-Chorus] I'm the bad guy [Verse 2] I like it when you take control Even if you know that you don't Own me, I'll let you play the role I'll be your animal My mommy likes to sing along with me But she won't sing this song If she reads all the lyrics She'll pity the men I know You might also like[Chorus] So you're a tough guy Like it really rough guy Just can't get enough guy Chest always so puffed guy I'm that bad type Make your mama sad type Make your girlfriend mad tight Might seduce your dad type I'm the bad guy Duh [Post-Chorus] I'm the bad guy, duh I'm only good at bein' bad, bad [Bridge] I like when you get mad I guess I'm pretty glad that you're alone You said she's scared of me? I mean, I don't see what she sees But maybe it's 'cause I'm wearing your cologne [Outro] I'm a bad guy I'm, I'm a bad guy Bad guy, bad guy I'm a bad345Embed"
2019,62nd Annual GRAMMY Awards (2019),Record Of The Year,"Hey, Ma",Bon Iver,True,"57 ContributorsHey, Ma Lyrics[Verse 1] I waited outside I took it remote I wanted a bath ""Tell the story or he goes"" ""Tell the story or he goes"" [Chorus] Full time, you talk your money up While it's living in a coal mine Tall time to call your Ma Hey Ma, hey Ma Tall vote, you know you mope it up Well, you wanted it your whole life You're back and forth with light [Verse 2] I waited outside I was tokin' on dope I hoped it all wunt go in a minute With the past that you know I wanted all that mind sugar I want it all mine I had a heavy mind, sugar I took it, it was right You might also like[Chorus] Full time, you talk your money up While it's living in a coal mine Tall time to call your Ma Hey Ma, hey Ma Tall vote, you know you mope it up Well, you wanted it your whole life You're back and forth with light [Interlude] You're back and forth You're back and forth You go back and forth And then you're back and forth And back and forth [Verse 3] I waited outside Then you took me in the room And you offered up the truth My eyes crawling up the window to the wall From dusk 'til dawn Let me talk to 'em Let me talk to 'em all [Chorus] Full time, you talk your money up While it's living in a coal mine Tall time to call your Ma Hey Ma, hey Ma Tall vote, you know you mope it up Well, you wanted it your whole life You're back and forth with light12Embed"
2019,62nd Annual GRAMMY Awards (2019),Record Of The Year,7 rings,Ariana Grande,True,"474 ContributorsTranslationsTürkçeEspañolRomânăPortuguêsPolskiItalianoHebrewΕλληνικάDeutschFrançaisNederlandsDanskفارسی7 rings Lyrics[Verse 1] Yeah, breakfast at Tiffany's and bottles of bubbles Girls with tattoos who like getting in trouble Lashes and diamonds, ATM machines Buy myself all of my favorite things (Yeah) Been through some bad shit, I should be a sad bitch Who woulda thought it'd turn me to a savage? Rather be tied up with calls and not strings Write my own checks like I write what I sing, yeah (Yeah) [Pre-Chorus] My wrist, stop watchin', my neck is flossy Make big deposits, my gloss is poppin' You like my hair? Gee, thanks, just bought it I see it, I like it, I want it, I got it (Yeah) [Chorus] I want it, I got it, I want it, I got it I want it, I got it, I want i